# Deep Learning
**Multilayer Perceptron (MLP)**: In this homework you are required to implement and train a 3-layer neural network to classify images of hand-written digits from the MNIST dataset. The input to the network will be a 28 × 28-pixel image, which is converted into a 784-dimensional vector. The output will be a vector of 10 probabilities (one for each digit). Specifically, the network you create should implement a function $g: \mathbb{R}^{784} \rightarrow \mathbb{R}^{10}$, where:

$$\mathbf{z}_{1} = \mathbf{W}^{(1)}\mathbf{x} + \mathbf{b}^{(1)}$$
$$\mathbf{h}_1 = ReLU(\mathbf{z}_1)$$
$$\mathbf{z}_2 = \mathbf{W}^{(2)}\mathbf{h}_1 + \mathbf{b}^{(2)}$$
$$\hat{\mathbf{y}} = g(\mathbf{x}) = Softmax(\mathbf{z}_2)$$

**Forward Propagation**: Compute the intermediate outputs $\mathbf{z}_{1}$, $\mathbf{h}_{1}$, $\mathbf{z}_{2}$, and $\hat{\mathbf{y}}$ as the directed graph shown below:

![jupyter](./img/mlp.jpg)

**Loss function**: After forward propagation, you should use the cross-entropy loss function: 
$$ f_{CE}(\mathbf{W}^{(1)},\mathbf{b}^{(1)}, \mathbf{W}^{(2)}, \mathbf{b}^{(2)}) =  - \frac{1}{n}\sum_{i=1}^{n} \sum_{k=1}^{10} \mathbf{y}_k^{(i)} \log \hat{\mathbf{y}}_k^{(i)} $$
where $n$ is the number of examples.

**Backwards Propagation**: To train the neural network, you should use stochastic gradient descent (SGD). 

# Question 1:
Compute the individual gradient for each term

$$\hat{y} = softmax\{w^{(2)}[ReLu(W^{(1)}x + b^{(1)})] + b^{(2)}\}$$

$$ \frac{\partial f_{CE}}{\partial \mathbf{W}^{(2)}} = \frac{\partial f_{CE}}{\partial \hat{y}} \frac{\partial \hat{y}}{\partial z_2} \frac{\partial z_2}{\partial W^{(2)}}= (\hat{y} - y)h_1^T   $$


$$ \frac{\partial f_{CE}}{\partial \mathbf{b}^{(2)}} = \frac{\partial f_{CE}}{\partial \hat{y}} \frac{\partial \hat{y}}{\partial z_2} \frac{\partial z_2}{\partial b^{(2)}} = \frac{1}{n} \sum_{i = 1}^n({\hat{y}^{i} - y})  $$


$$ \frac{\partial f_{CE}}{\partial \mathbf{W}^{(1)}} = \frac{\partial f_{CE}}{\partial z_2} \frac{\partial z_2}{\partial h_1} \frac{\partial h_1}{\partial z_1} \frac{\partial z_1}{\partial W^{(1)}}= \frac{1}{n} \sum_{i = 1}^n({\hat{y}^{i} - y}) W^{(2)} ReLu'(z_1) x$$
    
    
$$ \frac{\partial f_{CE}}{\partial \mathbf{b}^{(1)}} = \frac{\partial f_{CE}}{\partial z_2} \frac{\partial z_2}{\partial h_1} \frac{\partial h_1}{\partial z_1} \frac{\partial z_1}{\partial b^{(1)}} = \frac{1}{n} \sum_{i = 1}^n({\hat{y}^{i} - y}) W^{(2)} ReLu'(z_1)$$

# Question 2: 
Implement stochastic gradient descent for the network shown above in the *Starter Code* Below

# Question 3: 
Verify that your implemented gradient functions are correct using a numerical derivative approximation in *scipy.optimize.check_grad*

See the call to check grad in the starter code. 

Note that: the discrepancy should be less than 0.01.

# Question 4: 
Train the network using proper hyper-parameters (batch size, learning rate etc), and report the train accuracy and test accuracy in the *Starter Code* Below


**NOTE THAT**: You only need to submit this '.ipynb' file.

# Starter Code: 

In [60]:
import numpy as np
import scipy.optimize
import math

NUM_INPUT = 784  # Number of input neurons
NUM_HIDDEN = 50  # Number of hidden neurons
NUM_OUTPUT = 10  # Number of output neurons
NUM_CHECK = 5  # Number of examples on which to check the gradient
n = NUM_CHECK

# batch_size, learning rate, max_epoch are hyper-parameters
batch = 16
learning_rate = 0.02
max_epoch = 2100


# Given a vector w containing all the weights and biased vectors, extract
# and return the individual weights and biases W1, b1, W2, b2.
# This is useful for performing a gradient check with check_grad.

# unpack function will relieve w into W1, b1, W2, and b2 
def unpack (w):
    W1 = w[0:NUM_INPUT *NUM_HIDDEN].reshape(NUM_INPUT, NUM_HIDDEN)
    b1 = w[NUM_INPUT *NUM_HIDDEN:NUM_INPUT *NUM_HIDDEN + NUM_HIDDEN].reshape(NUM_HIDDEN, 1)
    W2 = w[NUM_INPUT *NUM_HIDDEN + NUM_HIDDEN : NUM_INPUT *NUM_HIDDEN + NUM_HIDDEN + NUM_HIDDEN * NUM_OUTPUT].reshape(NUM_HIDDEN, NUM_OUTPUT)
    b2 = w[NUM_INPUT *NUM_HIDDEN + NUM_HIDDEN + NUM_HIDDEN * NUM_OUTPUT: NUM_INPUT *NUM_HIDDEN + NUM_HIDDEN + NUM_HIDDEN * NUM_OUTPUT + NUM_OUTPUT].reshape(NUM_OUTPUT, 1)
    return W1, b1, W2, b2

# Given individual weights and biases W1, b1, W2, b2, concatenate them and
# return a vector w containing all of them.
# This is useful for performing a gradient check with check_grad.

# pack function will squeeze the W1, b1, W2, and b2 into a one-dim array
def pack (W1, b1, W2, b2):
    w = np.concatenate((W1.flatten(), b1.flatten(), W2.flatten(), b2.flatten()), axis=0)
    return w

# Load the images and labels from a specified dataset (train or test).
def loadData (which):
    images = np.load("data/mnist_{}_images.npy".format(which))
    labels = np.load("data/mnist_{}_labels.npy".format(which))
    return images, labels

# this softmax can be applied to matrix w, if we give the size of it (it means how many times of softmax we need to do to each line(it is an array))
def softmax(w, size):
    for i in range(size):
        w[i] = np.exp(w[i] - w[i].max())
        w[i] = w[i] / w[i].sum()
    return w

# Given training images X, associated labels Y, and a vector of combined weights
# and bias terms w, compute and return the cross-entropy (CE) loss. You might
# want to extend this function to return multiple arguments (in which case you
# will also need to modify slightly the gradient check code below).


# check whether the classifier correctly classified the instance X
def validation(X, Y, w):
    W1, b1, W2, b2 = unpack(w)
    X = X.reshape(1, X.size)
    Y = Y.reshape(1, Y.size)
    z1 = np.matmul(W1.T, X.T) + b1
    h1 = softmax(z1, 0)
    z2 = np.matmul(W2.T, h1) + b2
    y_hat = softmax(z2.T, 1)
    pred = 0
    label = 0
    max_prob = 0
#     go through the 10 positions
    for i in range(10):
        if y_hat[0][i] > max_prob:
            pred = i
            max_prob = y_hat[0][i]
#             this is one hot encoding
        if Y[0][i] == 1:
            label = i
    if (pred == label):
        return True
    else:
        return False

#     implement of fCE
def fCE (X, Y, w, instances):
#     calculate the predicted result
    W1, b1, W2, b2 = unpack(w)
    z1 = np.matmul(W1.T, X.T) + b1
    h1 = np.maximum(z1, 0)
    z2 = np.matmul(W2.T, h1) + b2
    y_pred = softmax(z2.T, instances)
    cost = 0
#     calculate the cost
    for i in range(instances):
        for j in range(NUM_OUTPUT):
            cost += Y[i][j] * math.log(y_pred[i][j])
    cost *= (-1 / instances)
    return cost

# Given training images X, associated labels Y, and a vector of combined weights
# and bias terms w, compute and return the gradient of fCE. You might
# want to extend this function to return multiple arguments (in which case you
# will also need to modify slightly the gradient check code below).
def gradCE (X, Y, w, instances):
#     forward
    W1, b1, W2, b2 = unpack(w)
    z1 = np.matmul(W1.T, X.T) + b1
    h1 = np.maximum(z1, 0)
    z2 = np.matmul(W2.T, h1) + b2
    y_pred = softmax(z2.T, instances)
    
#     begin backward
    delta = y_pred - Y
    dw2 = 1 / instances * np.dot(delta.T, h1.T).T
    db2 = 1 / instances * delta.T.sum(axis = 1)
    dh1 = 1 / instances * np.dot(W2, delta.T)
#     calcutate relu's gradient
    for i in range(z1.shape[0]):
        for j in range(z1.shape[1]):
            if z1[i][j] < 0:
                z1[i][j] = 0
            else:
                z1[i][j] = 1
    dz1 = dh1 * z1
    db1 = dz1.sum(axis = 1)
    dw1 = np.dot(dz1, X).T
    grad = pack(dw1, db1, dw2, db2)
    return grad

# Given training and testing datasets and an initial set of weights/biases b,
# train the NN.
## return the train accuracy and the test accuracy

# training function
def train (trainX, trainY, testX, testY, w):
    # W1, b1, W2, b2 = unpack(w)
    train_acc = 0
    test_acc = 0
    epoch = 0
    loss = 1e3
    w_updated = w
    
#     set the ending criterion
    while(loss > 0.001 and epoch < max_epoch):
        randState = np.random.get_state()
        np.random.shuffle(trainX)
        np.random.set_state(randState)
        np.random.shuffle(trainY)
        loss=fCE(trainX[0:batch], trainY[0:batch], w_updated, batch)
        w_updated=w_updated-gradCE(trainX[0:batch], trainY[0:batch], w_updated,batch) * learning_rate
        epoch += 1
#         print loss to check its trand
        print(loss)
    train_corr=0
    for i in range(0,trainX.shape[0]):
        if(validation(trainX[i],trainY[i],w_updated)):
            train_corr+=1
    train_acc=train_corr / trainX.shape[0]
    
    test_corr=0
    for i in range(0,testX.shape[0]):
        if(validation(testX[i],testY[i],w_updated)):
            test_corr+=1
    test_acc=test_corr / testX.shape[0]
    
    print("Training Accuracy",train_acc,"\tTest Accuracy",test_acc)
    return train_acc, test_acc

if __name__ == "__main__":
    # Load data
    trainX, trainY = loadData("train")
    testX, testY = loadData("test")
    
# print the shape to check if the multiply of matrix is correct
#     print(trainX.shape)(10000, 784)
#     print(trainY.shape)(10000, 10)
#     print(testX.shape)(5000, 784)
#     print(testY.shape)(5000, 10)
    
    # Initialize weights randomly
    W1 = 2*(np.random.random(size=(NUM_INPUT, NUM_HIDDEN))/NUM_INPUT**0.5) - 1./NUM_INPUT**0.5
    b1 = 0.01 * np.ones(NUM_HIDDEN)
    W2 = 2*(np.random.random(size=(NUM_HIDDEN, NUM_OUTPUT))/NUM_HIDDEN**0.5) - 1./NUM_HIDDEN**0.5
    b2 = 0.01 * np.ones(NUM_OUTPUT)
    w = pack(W1, b1, W2, b2)
    
    
    # Check that the gradient is correct on just a few examples (randomly drawn).
    idxs = np.random.permutation(trainX.shape[0])[0:NUM_CHECK]
    discrepancy = scipy.optimize.check_grad(lambda w_: fCE(np.atleast_2d(trainX[idxs,:]), np.atleast_2d(trainY[idxs,:]), w_, NUM_CHECK), \
                                    lambda w_: gradCE(np.atleast_2d(trainX[idxs,:]), np.atleast_2d(trainY[idxs,:]), w_, NUM_CHECK), \
                                    w)
    if discrepancy < 0.01:
        print("My implemented cost and gradient functions are correct")

My implemented cost and gradient functions are correct


In [62]:
# Train the network and return the train accuracy and test accuracy
# We seperate gradient and training, to make the process clearer
train_acc, test_acc = train(trainX, trainY, testX, testY, w)

2.304533412247085
2.2814914491338008
2.3252081454007025
2.3049871537014908
2.2635928214887864
2.2699339128663585
2.252785268556087
2.255230424725228
2.271384561645785
2.298296004452692
2.3032962595284348
2.294406797632306
2.2466143757963897
2.239769184253707
2.312513447472053
2.274657289846844
2.2722083541102385
2.261778413121961
2.2110188651098377
2.2623928858376736
2.211905803335363
2.210396610202834
2.217941779781451
2.2297876411037283
2.2224357926833957
2.1321176240400095
2.1675974065124564
2.1124401694904638
2.1407772953100186
2.2148249255947867
2.2334349748797457
2.1905376918834336
2.09134999355519
2.1421259963784554
2.2019951662800907
2.190030020503124
2.178047575545982
2.1198234505105615
2.2004643589261197
2.2115415215896426
2.0853928864261992
2.103005685785059
2.016772860551242
2.043343485844071
2.042765614199448
2.1098041523214612
2.1332682575022255
2.1073591125765656
2.182483077023219
2.167161882219621
1.9933207767518069
1.9796958206314808
2.139147368236052
1.945775647007211

0.6581894918454152
0.5730007406464721
0.8544647460674062
0.6132511371831305
0.5963505589666368
0.7452406086418293
0.7185127887112315
0.45792964305579714
0.7991990083470161
0.7542219879655976
0.5863029378242538
0.6589927025826727
0.6498172563769328
0.4639832257284825
0.4623787544808848
0.8474168090623394
0.6481192154608794
0.34726486472648427
0.3913927970487212
0.7816457137849528
0.8564750700013534
0.7264341958639062
0.6152955675086117
0.578341742025508
0.7256640716726492
0.5735392445107187
1.019767253749577
0.7106498576014249
0.6042112196987862
0.3171273791539414
0.6241936382517534
0.6091084000778845
0.8191258643208007
0.5410925130031173
0.7085730306497149
0.6525807390505184
0.5658520216327436
0.5444087857621378
0.4899357560700672
0.7911934296729416
0.3805475879906003
0.3298362003401545
0.8141949400626478
0.6141751799264346
0.6789817584214696
0.5485873732532207
0.758640499767531
0.6493898563009521
0.542869877599087
0.5060139065064022
0.6597446105876371
0.5864716566947985
0.693233533664

0.5052410474251753
0.22556137701265197
0.5003932537568114
0.11764901180799395
0.49057372377142294
0.5039313689305525
0.31997608459301674
0.40003284871742417
0.419575352470884
0.20259241307066872
0.20983703810004178
0.5458437859043774
0.4295027443958439
0.3590431849811869
0.5064797196192122
0.36613527865619516
0.36541334029630596
0.3589718659959146
0.3334868283601452
0.6838156737067431
0.299495209999416
0.3345890329806484
0.12979875207374536
0.6112452402934716
0.27507124346622414
0.4822765384408014
0.4557449946664194
0.6058921711324394
0.11148414676413007
0.5090636087182935
0.25827841210841085
0.43927783444931306
0.5518588354437954
0.24738633343234767
0.3692498914091658
0.8547523677370319
0.4344211661078901
0.46819971272409444
0.32415685321793586
0.9692368553694496
0.6315517944477278
0.5189216320677978
0.5472252758690965
0.29055242645221163
0.5577746707199834
0.48041161039125363
0.2555789054675949
0.37066319456603036
0.385611541006027
0.5932427704469022
0.3000491556635118
0.554513831612

0.36581219276215143
0.49999543280175796
0.5891987468279813
0.416635129618241
0.3760977441305251
0.27237900811028265
0.5366919360644975
0.511151455788229
0.2581584867213559
0.3020389318940295
0.31620917891401323
0.39694895939019514
0.19820983999254566
0.23202892848275009
0.40646520069010544
0.35201644412077093
0.1552795081548652
0.11862129322533144
0.39444364922524944
0.5659376562025555
0.3803861080280001
0.33369207238888376
0.21107486738533787
0.5529609326372683
0.1205048204747708
0.1405693289124724
0.4967075270077077
0.47619907789760935
0.200064975801688
0.4219691830675488
0.33647292128678885
0.314420049530713
0.6224949485381922
0.1083193208350887
0.3868699102334447
0.38842182922005575
0.2737974496264544
0.08176628434464599
0.46462238959600605
0.569119577222164
0.2970132707201215
0.5066740012817024
0.3253916900512625
0.2869881854678051
0.5571131877526111
0.12318097921822733
0.13146327310702774
0.2568277859699261
0.3468398127033877
0.3750723197399753
0.6551000420079878
0.51411575229593

0.27488993547041785
0.6460230749035752
0.19861918292648614
0.5708976921257493
0.2040786561042999
0.23241317615578355
0.32714440639411835
0.1068617999431749
0.2644758854631767
0.29017300959634834
0.5746973639086648
0.07107650039348137
0.32285290939248407
0.37199022897488204
0.29142070400040304
0.6684538672513571
0.12692436215618252
0.17051904432044573
0.3946686087517717
0.23196010016950805
0.6700648543455204
0.05784401821374396
0.10227050522447056
0.28241715135078205
0.40921889281301027
0.34925483210584224
0.1821997084667526
0.44975204488681403
0.5617985007662957
0.2919992944734775
0.07971057783486424
0.21569850262290025
0.23937910806682622
0.24789049431685475
0.3775984747137647
0.08366422909455024
0.16513346192950018
0.19015520514417092
0.19934394475013434
0.1527404789177535
0.17234619843822063
0.17271328097591226
0.34901973407936626
0.35621531209789603
0.20135900621281458
0.6882701938683362
0.3158390947422651
0.09914101015836402
0.09080626376391356
0.09905651149848874
0.19401121934274